# Empleabilidad, Matrícula y Titulación: Estudiar Minería en Chile
*Por Gonzalo Nelis S.*

Este es un breve análisis de la oferta en educación superior de la carera de Ingeniería de Minas. Todos los datos fueron extraídos de la base de datos pública de la página [Mi Futuro](www.mifuturo.cl) a cargo del Ministerio de Educación. La idea es entender como ha cambiado el panorama de estudiar Ingeniería en Minas en los ultimos años.

## ¿Qué carreras se consideraron?

No todas las universidades ofrecen la misma carrera. Hay ingenierias en ejecución, civil, a secas, etc. En este análisis, se consideraron todas las carreras que en el campo de "NOMBRE CARRERA" de la base de datos incluyeran "Ingeniería" y "Minas" a la vez. Se incluyen todas las menciones, programas especiales, cambios de malla que caigan en esta descripción. Programas de Magister y Doctorado también, si es que caen en dicha categoria, aunque su aporte es bastante menor por lo que no afectarán las conclusiones. No se incluyen carreras de metalurgia de forma exclusiva, ni tampoco carreras técnicas (el titulo debe incluir "Ingeniería). Esto no quita que institutos profesionales sí puedan dictar la carrera de Ingeniería, por lo que se incluyen en el análisis.


In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

## Titulados

La primera parada de este análisis es revisar cuánta gente (y de donde) se está titulando. Existe la idea generalizada que la profesión de la ingeniería en minas está saturada, y que hay muchos profesionales saliendo al campo laboral. ¿Será verdad?

Bueh, sí lo es:

In [14]:
titulados = pd.read_csv("Data/Titulacion.csv", delimiter = ';')
titulados["AÑO"] = titulados["AÑO"].str.extract(r'(\d+)')
con_minas = titulados[(titulados["NOMBRE CARRERA"].str.contains(r"\b(INGENIERIA|INGENIERÍA)\b.*\bMINAS\b"))]

tipo = pd.DataFrame(con_minas.groupby(["AÑO","CLASIFICACIÓN INSTITUCIÓN NIVEL 3"])["TOTAL TITULADOS"].agg(['sum','count'])).reset_index()
fig = px.bar(tipo, x="AÑO", color="CLASIFICACIÓN INSTITUCIÓN NIVEL 3", y="sum", title="Titulados por Institución de Origen")
fig.update_layout(yaxis={'title': 'Titulados'},
                    legend_title_text='Institución')
fig.show()

El 2007 hubo un total de 178 titulados de alguna carrera afín a la ingeniería de minas. ¿El 2019? **1517**. Un ligero aumento del **850%**. Y por la tendencia del grafico, aún no llegamos al peak (los matriculados nos dan una idea de cuando será ese peak, así que a seguir leyendo!). El desglose por institución es interesante, también:

- Las universidades estatales pasaron de 112 en 2007 a 398 titulados en 2019. Casi 400%
- Los institutos profesionales pasaron de 66 a 703. **Un 1000% de crecimiento**.
- Las Universidades privadas no tradicionales: 0 en 2017 --> 326 en 2019.
- Las privadas tradicionales recién aparecieron con titulados el 2017, y representan un porcentaje menor en todos los años.

¿Cómo pasamos de 178 a 1517 titulados en 12 años? Bueno, hubo una explosión de instituciones que abrieron la carrera. ¿Cuántas? Acá va el gráfico:

In [15]:
sin_duplis = con_minas.drop_duplicates(subset=["AÑO","NOMBRE INSTITUCIÓN","CÓDIGO CARRERA"])
tipo = pd.DataFrame(sin_duplis.groupby(["AÑO","CLASIFICACIÓN INSTITUCIÓN NIVEL 3"])["TOTAL TITULADOS"].agg(['sum','count'])).reset_index()
fig = px.bar(tipo, x="AÑO", color="CLASIFICACIÓN INSTITUCIÓN NIVEL 3", y="count", title="Número de Carreras con Titulados")
fig.update_layout(yaxis={'title': 'Numero de Carreras'},
                    legend_title_text='Institución')
fig.show()

Antes que todo, una aclaración: este gráfico está basado en el "Codigo de carrera" que asigna el Ministerio de Educación. Este codigo es único para cada versión de la carrera. Eso quiere decir que una misma Universidad puede presentar varios codigos únicos: ¿La misma carrera en distintas sedes? codigos distintos. ¿Cambio de malla? codigos distintos. ¿Diurno/vespertino? Codigos distintos. Así que la forma correcta de interpretar este gráfico es pensar en "Numero de formas en que me puedo titular de una carrera de ingenieria de minas". 

El número era mas o menos estable, hasta que se disparó por la aparición de Universidades privadas. El 2018 fue el peak, con 56 carreras relacionadas a la Ing. de Minas en instituciones privadas no tradicionales. En verdad, no hay 56 Universidades Privadas dando la carrera, pero es notable la capacidad de armar carreras en distintos formatos. La tabla que sigue muestra el número de carreras de cada Universidad:

In [16]:

con_minas[(con_minas["AÑO"]=='2018') & (con_minas["CLASIFICACIÓN INSTITUCIÓN NIVEL 3"]=="Universidades Privadas")].groupby("NOMBRE INSTITUCIÓN")["AÑO"].count().to_frame().rename(columns = {"AÑO":"Carreras"})

,Carreras
NOMBRE INSTITUCIÓN,
UNIVERSIDAD ANDRES BELLO,3
UNIVERSIDAD BOLIVARIANA,3
UNIVERSIDAD DE ACONCAGUA,11
UNIVERSIDAD DE LAS AMERICAS,9
UNIVERSIDAD DE VIÑA DEL MAR,1
UNIVERSIDAD LA REPUBLICA,16
UNIVERSIDAD LOS LEONES,2
UNIVERSIDAD PEDRO DE VALDIVIA,10
UNIVERSIDAD SAN SEBASTIAN,1


Por ejemplo, la Universidad Andrés Bello tuvo titulados de 3 carreras: Civil en Minas de Concepción, Viña y Santiago. Todo calza. 

Otro ejemplo: La U. Pedro de Valdivia tuvo 10. Solo en el año 2018. ¿De donde sale tanta carrera? Las enumero:

- Civil en Minas: Santiago, La Serena, Antofagasta (Diurno/Vespertino), Chillán.
- Ingeniería en Minas: Santiago (Diurno/Vespertino), La Serena, Antofagasta (Diurno/Vespertino) 

La Universidad La República tiene 16. La mayoría son planes de Continuidad de Estudios en distintos formatos dependiendo de la duración de la carrera de base.

## Matriculados

Los titulados nos explican el estado actual del campo laboral. Pero ¿cómo han variado las matriculas en la carrera? Da la idea de que ya no existe ese boom por estudiar minería. Acá va la data:

In [17]:
matriculados = pd.read_csv("Data/Matricula.csv", delimiter = ';')
matriculados["AÑO"] = matriculados["AÑO"].str.split('_', expand = True)[1]
matriculados_con_minas = matriculados[(matriculados["NOMBRE CARRERA"].str.contains(r"\b(INGENIERIA|INGENIERÍA)\b.*\bMINAS\b"))]
matriculados_por_tipo = pd.DataFrame(matriculados_con_minas.groupby(["AÑO","CLASIFICACIÓN INSTITUCIÓN NIVEL 3"])["TOTAL MATRICULADOS PRIMER AÑO"].agg(['sum','count'])).reset_index()
fig = px.bar(matriculados_por_tipo, x="AÑO", color="CLASIFICACIÓN INSTITUCIÓN NIVEL 3", y="sum", title="Total Matriculados por Institución de Origen")
fig.update_layout(yaxis={'title': 'Alumnos Matriculados'},
                    legend_title_text='Institución')
fig.show()


Como diría Omar Labruna [*La moda se acabó*](https://www.youtube.com/watch?v=Lv2Elt85N7Q). El peak es más que notable: casi 3500 matriculados durante 3 años seguidos. Las matriculas decayeron el año 2016. El precio del cobre bajó por ahí por el 2015, así que de nuevo, todo calza. O mejor dicho, "el mercado se regula solo". 

Interesante es también que la matrícula en universidades estatales igual va a la baja: 471 matriculados en 2020, casi en niveles pre-boom (424 en 2007).

Pero la mayor regulación la sufrieron las Universidades privadas: de 1127 en 2014, a 323 en 2020. De acuerdo a estos datos, la generación que se tituló en 2019 debería ser la de inicios del aumento de matricula (2013 aproximadamente). En base a eso, es esperable que sigamos con una cantidad alta de titulados por uno o dos años más, para luego empezar el decenso. Sin embargo, el numero de matriculados en 2013 es notablemente superior al numero de titulados del 2019. Esto puede indicar que la duración real de las carreras es de más de 6 años, o que existe una alta tasa de deserción (ambos numeros están en la base de datos de www.mifuturo.cl, por si alguien quiere hacer ese análisis).

El "mercado" no solo se ha "regulado" en el número de matriculados: también en el número de carreras disponibles:

In [18]:
fig = px.bar(matriculados_por_tipo, x="AÑO", color="CLASIFICACIÓN INSTITUCIÓN NIVEL 3", y="count", title="Sedes con matriculados")
fig.update_layout(yaxis={'title': 'Sedes'},
                    legend_title_text='Institución')
fig.show()

Ya no hay 50 o más carreras que imparten la ingeniería de minas. El numero tambien ha ido descendiendo. La falta de matrícula obliga a cerrar ciertas carreras (sedes, programas, etc.). Actualmente, la oferta de las Universidades privadas es notablemente menor:

In [19]:
matriculados_con_minas[(matriculados_con_minas["AÑO"]=='2020') & (matriculados_con_minas["CLASIFICACIÓN INSTITUCIÓN NIVEL 3"]=="Universidades Privadas")].groupby("NOMBRE INSTITUCIÓN")["AÑO"].count().to_frame().rename(columns = {"AÑO":"Carreras"})

,Carreras
NOMBRE INSTITUCIÓN,
UNIVERSIDAD ANDRES BELLO,3
UNIVERSIDAD BOLIVARIANA,2
UNIVERSIDAD CENTRAL DE CHILE,2
UNIVERSIDAD DE ACONCAGUA,6
UNIVERSIDAD DE LAS AMERICAS,9
UNIVERSIDAD DE VIÑA DEL MAR,1
UNIVERSIDAD LA REPUBLICA,3
UNIVERSIDAD LOS LEONES,1
UNIVERSIDAD PEDRO DE VALDIVIA,10


Comparado a nuestro analisis de los titulados en 2018, los cambios notables: 
- Universidad de La República: de 16 a 3.
- Universidad de Aconcagua: de 11 a 6.

## Empleabilidad

La pieza final del puzzle es la empleabilidad. Pero es la más dificil de analizar: la data es tremendamente escasa. No está la base de datos completa por carrera y por año, solo porcentajes de ciertas universidades. El portal de [Mi Futuro](https://www.mifuturo.cl/buscador-de-estadisticas-por-carrera/) provee algunos datos agrupados que copio acá por completitud. Al buscar "Empleabilidad" para "Ingenieria Civil de Minas", aparece esto:


In [20]:
import plotly.graph_objects as go
years=list(range(2015,2020))

emp_1 = [95.4, 87.1, 83.0, 83.5, 83.4]
emp_2 = [96.3, 95.8, 92.4, 91.0, 91.5]
fig = go.Figure(data=[
    go.Bar(name='1er año', x=years, y=emp_1, text=[f'{num}' for num in emp_1], textposition='auto'),
    go.Bar(name='2do año', x=years, y=emp_2, text=[f'{num}' for num in emp_2], textposition='auto')
])
# Change the bar mode
fig.update_layout(barmode='group', title="Empleabilidad - Ing. Civil de Minas", yaxis = {'title':"Empleabilidad (%)"})
fig.show()

A la baja, sí, pero aún muy alta, lo que es bueno. Ahora, si buscamos el panorama de "Ingenieria en Minas y Metalurgia" en Universidades, el panorama cambia un poco:

In [21]:
import plotly.graph_objects as go
years=list(range(2015,2020))
emp_1 = [83.1, 77.7, 69.1, 69.0, 75.3]
emp_2 = [92.2, 88.4, 84.1, 86.0, 78.5]
fig = go.Figure(data=[
    go.Bar(name='1er año', x=years, y=emp_1, text=[f'{num}' for num in emp_1], textposition='auto'),
    go.Bar(name='2do año', x=years, y=emp_2, text=[f'{num}' for num in emp_2], textposition='auto')
])
# Change the bar mode
fig.update_layout(barmode='group', title="Evolución de Empleabilidad - Ing. en Minas y Metalurgia (U)", yaxis = {'title':"Empleabilidad (%)"})
fig.show()

Acá el descenso está un poco mas marcado. La categoría de Ing. en Minas y Metalurgia agrupa todo lo que no sea Ing. Civil: Ejecución en Minas e Ing. de Minas "a secas". Si ahora buscamos esta misma categoria, pero en institutos profesionales, el panorama es el siguiente:

In [22]:
years=list(range(2015,2020))
emp_1 = [79.2, 71.9, 70.6, 63.9, 77.9]
emp_2 = [87.4, 80.8, 82.0, 77.9, 67.8]
fig = go.Figure(data=[
    go.Bar(name='1er año', x=years, y=emp_1, text=[f'{num}' for num in emp_1], textposition='auto'),
    go.Bar(name='2do año', x=years, y=emp_2, text=[f'{num}' for num in emp_2], textposition='auto')
])
# Change the bar mode
fig.update_layout(barmode='group', title="Evolución de Empleabilidad - Ing. en Minas y Metalurgia (IP)", yaxis = {'title':"Empleabilidad (%)"})
fig.show()

Más baja aún. Todos estos datos hacen pensar que la mejor apuesta es estudiar "Ing Civil en Minas", pues posee la mejor empleabilidad. Esto es cierto, pero hay que tener algunso resguardos con estos datos. De la metodología que se usó para obtener los datos de [Empleabilidad](https://www.mifuturo.cl/wp-content/uploads/2020/11/Metodologia-Buscador-Empleabilidad-e-Ingresos_2020_2021_SIES.pdf):

- Empleabilidad al 1er año: Corresponde al porcentaje de titulados de las cohortes 2016, 2017 y 2018 que, teniendo información sobre ingresos,obtuvieron ingresos iguales o superiores al sueldo mínimo en el primer año después de su titulación, es decir en los años de ingresos 2017,2018 y 2019 respectivamente. Solo se informa este dato para aquellas carreras de instituciones que contaban con al menos 25 casos válidos en la sumade tres cohortes o 50 casos en la suma de dos cohortes.

O sea, carreras con pocos titulados no cuentan para este gráfico. Por otro lado, no se cuenta si efectivamente está ejerciendo la profesión de la Ingeniería: solo se registra si en la declaración de impuestos al SII se cuenta con ingresos superiores al sueldo mínimo. Esto podría explicar, en parte, la caída brusca del sueldo promedio: 

In [23]:
years=list(range(2015,2020))
sueldo = [4672913, 3280146, 3115344, 2956856, 2701238]
sueldo_str = [f'${num:,}' for num in sueldo]
fig = go.Figure(data=[
    go.Bar(name='1er año', x=years, y=sueldo, text=sueldo_str, textposition='auto')
])
# Change the bar mode
fig.update_layout(barmode='group', title="Sueldo promedio (4to año) - Ing. Civil en Minas (U)", yaxis = {'title':"Pesos"})
fig.show()

No todos los egresados están trabajando con los sueldos estratosféricos que ofrecía LUN. Aún están totalmente por sobre el promedio, eso sí, así que al parecer la industria sigue pagando bien. Solo que el trabajo no alcanza para todos.

Lo último: Aunque la opción de Ing. Civil parece atractiva en base a empleabilidad, el mercado laboral sigue siendo tremendamente selectivo. **Al parecer**, no le da lo mismo de donde viene el egresado. Acá hay súper pocos datos, así que a tener cuidado con las interpretaciones. Existen un par de instituciones donde sí se reportan sus numeros de empleabilidad desglosados:

In [24]:
cols_to_import = []
empleabilidad = pd.read_csv("Data/Empleabilidad.csv", delimiter = ';').fillna("n/a")
con_minas = empleabilidad[empleabilidad["Nombre carrera (del título)"].dropna().str.contains(r"\b(Civil)\b.*\b(Minas|minas)\b")]
con_minas.iloc[:,[2,6,10,11]]


,Nombre de institución,Nombre carrera (del título),Empleabilidad 1er año,Empleabilidad 2° año
647,Universidad Andrés Bello,Ingeniería Civil en Minas,50.8%,n/a
915,Universidad de Atacama,Ingeniería Civil en Minas,80.0%,88.8%
937,Universidad de Chile,Ingeniería Civil en Minas,93.8%,97.0%
1045,Universidad de La Serena,Ingeniería Civil en Minas,77.9%,87.5%
1157,Universidad de Santiago de Chile,Ingeniería Civil en Minas,85.3%,91.5%
1407,Universidad Pedro de Valdivia,Ingeniería Civil en Minas,49.5%,76.3%


De nuevo, **hay pocos datos**, pero en dos universidades privadas no tradicionales, la empleabilidad es notoriamente más baja en comparación a las Universidades tradicionales para Ing. Civil en Minas. En ese sentido, hay una labor grande por parte de estas instituciones de posicionar a sus egresados y darle las herramientas para que se puedan diferenciar del resto.

## Al cierre

Los datos son bien elocuentes. La empleabilidad va cayendo, los sueldos tambien. La matrícula se disparó, y no porque el mercado laboral pidiera esa cantidad de ingenieros. Es cierto que la oferta de carreras y la matrícula se han regulado para hacer frente a la baja en empleabilidad. Pero la regulación no es instantanea: conlleva cientos (o miles) de jóvenes que no pueden ejercer la carrera que estudiaron pues el mercado laboral no da abasto. Y de seguro que pasa con otras carreras, también.

No todo es gris, eso sí: la minería sigue teniendo un montón de desafíos a futuro. Va a seguir siendo parte fundamental del desarrollo de Chile y el mundo. Se van a incorporar nuevas tecnologías. Se tendrá que hacer mas sustentable. Para que todo eso ocurra se necesitan ingenieros brillantes, que lideren esos cambios. Y se necesitan instituciones de educación superior que brinden esas herramientas. Y quien sabe, quizás, con seriedad, podamos convertirnos en exportadores de mano de obra avanzada para otras partes del mundo [donde la matrícula va a la baja](https://www.abc.net.au/news/2020-12-16/mining-companies-hire-international-engineers-in-skills-shortage/12986650). 

## Contacto
¿Me equivoqué en un dato y quieres arreglarlo? ¿Quieres hacer otros análisis? Pueden encontrar las bases de datos y este notebook en [Github/gnelis](https://github.com/gnelis/empleabilidad).